# Cavity Quantum Electrodynamics Hartree-Fock Self-Consistent Field Theory

"""Tutorial implementing a CQED-RHF program"""

__authors__ = "J. McTague, J. Foley, A. E. DePrince III"
__credits__ = "J. McTague, J. Foley, A. E. DePrince III"
__email__   = "foleyj10@wpunj.edu, deprince@fsu.edu"

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "11/04/2021"

## I. Theoretical Overview

- Introduce $\hat{H}_{PF} = \hat{H}_e + \hat{H}_p + \hat{H}_{ep} + \hat{H}_{dse}$ and define all terms as in Eq. 2-5 in [[McTague:2021:ChemRxiv](https://chemrxiv.org/engage/chemrxiv/article-details/611fa8d08a6faa13229c8be6)]
- Introduce HF Reference $|R\rangle = |\Phi_0\rangle |0\rangle$ and show that only the 
  $\langle R| \hat{H}_e + \hat{H}_{dse} | R \rangle$ terms contribute to the energy expectation value 
- Expand the $\hat{H}_{dse}$ to show all terms that contribute as in Eq. 8 in [[McTague:2021:ChemRxiv](https://chemrxiv.org/engage/chemrxiv/article-details/611fa8d08a6faa13229c8be6)], pointing out the 1- and 2-electron dipole and 1-electron quadrupole terms that arise
- Show Augmented Fock matrix and CQED-RHF energy expression (Eqs. 10-13 in [[McTague:2021:ChemRxiv](https://chemrxiv.org/engage/chemrxiv/article-details/611fa8d08a6faa13229c8be6)])
- Explain how ordinary SCF routine can be used to solve CQED-RHF equations with slight modification that $\langle \mu \rangle$ should be transformed to current CQED-RHF orbital basis at each iteration
  




# I. Theoretical Overview

The Pauli-Fierz Hamiltonian, $\hat{H}_{PF}$, is defined in the dipole approximation as:
$$\hat{H}_{PF} = \hat{H}_e + \hat{H}_p + \hat{H}_{ep} + \hat{H}_{dse}\tag{1}$$
***
### *Electronic Term*



The term $\hat{H}_e$ represents the electronic Hamiltonian and is given through:
$$\hat{H}_e = \sum^{N_e}_{i} \hat{T}_e(x_i) + \sum^{N_e}_i \sum^{N_N}_A \hat{V}_{eN}(x_i :X_A) + \sum^{N_e}_i \sum^{N_e}_j (\hat{V}_{ee}(x_i, x_j) + V_{N,N}\tag{2}$$
Where $\hat{T}_e(x_i)$ is the electronic kinetic operator for electron $i$, $\hat{V}_{eN}(x_i:X_A)$ is the attractive coulomb operator for electron $i$ and nucleus $A$, $\hat{V}_{ee}(x_i : x_J)$ is the repulsive coulomb operator for electrons $i$ and $j$, and $V_{N,N}$ is the total repulsive coulomb potential between all nuclei. When using the Born-Oppenheimer Approximation, the term $V_{N,N}$ is treated a constant. Additionally, we can neglect the nuclear kinetic energy, and the electron-nuclear attraction will depend upon the coordinates of the fixed nuclear coordinates.
***
### *Photonic Term*

The next term considered within the Pauli-Fierz Hamiltonian is the photonic Hamiltonian, $\hat{H}_p$, which is defined as:
$$\hat{H}_p = \omega\hat{b}^{\dagger{}}\hat{b}\tag{3}$$
Here, $\omega$ represents the frequency of a photon. 

***
### *Bilinear Coupling Term*
The third term in the Hamiltonian is a bilinear coupling term that describes coupling of the photonic and electronic degrees of freedom given by, $\hat{H}_{ep}$ and is defined as:
$$\hat{H}_{ep} = \sqrt{\frac{\omega}{2}}\left(\lambda \cdot (\hat{\mu} - \langle \mu \rangle )\right)\left(\hat{b}^{\dagger{}}+\hat{b}\right)\tag{6}$$

Where $\lambda$ describes the coupling strength, $\hat{\mu}$ is the dipole operator, and $\langle\mu\rangle$ is the molecular dipole expectation value in the ground state. Here, the expectation value has characteristic Cartesian components, denoted by $\xi$. A given dipole operator with $\xi$ components is defined through:
$$\hat{\mu}^{\xi} = \sum^{N_e}_i \hat{\mu}^\xi (x_i)+{\mu}_{nuc}^\xi \mid \xi \in \{x,y,z\}\tag{7}$$

Where $\mu^{\xi}(x_i)$ is defined as an operator that depends upon electronic coordinates, and $\mu_{nuc}^{\xi}$ is the nuclear dipole moment where the Cartesian components are treated as functions of the nuclear coordinate.

***
### *Dipole Self Energy Term*
The final component of the Hamiltonian, $\hat{H}_{dse}$ is the dipole self energy term, and is defined through:
$$\hat{H}_{dse} = \frac{1}{2}\left(\lambda \cdot (\hat{\mu} - \langle\mu\rangle)\right)^2\tag{8}$$

Here, $\lambda$ represents the polarization term, $\hat{\mu}$ represents the molecular dipole operator and $\langle\mu\rangle$ represents the ground state molecular expeectation value.
***

***
### *Hartree-Fock Reference*
Having discussed the terms that comprise our Hamiltonian, we can now begin our CQED procedure, which we initiate through the introduciton of a product wavefunction, $|R\rangle$. Here, we define $|R\rangle$ through the following: 
$$|R\rangle = |\Phi_{0}\rangle |0\rangle\tag{9}$$
Here, we have defined $|R\rangle$ as the product wavefunciton between an electronic Slater determinant, $\Phi_{0}$ and a zero-photon number state, $|0\rangle$. We can begin by performing a canonical RHF calculation to initialize the Slater determinant. 
With $|R\rangle$ defined, we can now apply Equation 9 to Equation 1, arriving at the following:
$$\langle R|\hat{H}_{ep}|R\rangle + \langle 0|\hat{H}_{p}|0\rangle + \langle \Phi_{0}| \hat{H}_{e} + \hat{H}_{dse}|\Phi_{0}\rangle\tag{10}$$

As we are using a zero-photon number state, we find that the $\hat{H}_{ep}$ and $\hat{H}_{p}$ terms both go to zero, leaving behind only the $\hat{H}_e$ and $\hat{H}_{dse}$. The resulting terms are seen in Equation 11.
$$\langle\Phi_{0}|\hat{H}_{e}|\Phi_{0}\rangle + \langle\Phi_{0}|\hat{H}_{dse}|\Phi_{0}\rangle\tag{11}$$

From this new equation, our CQED-RHF implementation differs from ordinary RHF via the dipole self energy term--as $\hat{H}_{e}$ provides the ordinary RHF energy. 
***

***
### Dipole Self Energy Term Expansion
In order to better understand the role of the dipole self energy term, we will look at its expansion. 

$$\hat{H}_{dse} = \sum_{\xi, \xi^{'}}\sum_{i,j>i}\lambda^{\xi}\lambda^{\xi^{'}}\hat{\mu}^{\xi}(x_i)\hat{\mu}^{\xi^{'}}(x_j) \\- \frac{1}{2}\sum_{\xi,\xi^{'}}\sum_{i}\lambda^{\xi}\lambda^{\xi^{'}}\hat{Q}^{\xi\xi^{'}}(x_i) \\+ (\lambda\cdot\mu_{nuc}-\lambda\cdot\langle\mu\rangle)\sum_{\xi}\sum_{i}\lambda^{\xi}\hat{\mu}^{\xi}(x_i) \\+ \frac{1}{2}(\lambda\cdot\mu_{nuc})^2-(\lambda\cdot\langle\mu\rangle)(\lambda\cdot\mu_{nuc}) + \frac{1}{2}(\lambda\cdot\langle\mu\rangle))^2\tag{12}$$

In this expansion of the dipolse self energy term, we observe several unique terms show up. We observe both 1- and 2-electron electronic dipole contributions, as well as 1-electron quadrupole contributions. The first two terms within this expansion effectively determine the presence of either the 2-electron dipole contribution or the 1-electron quadrupole contribution, given the respective values of the indices i and j. That is, we observe the 2-electron dipole contribution precisely when $i \neq j$, and the 1-electron quadrupole contribution when $i=j$. For clarification, we can rewrite $\hat{Q}^{\xi\xi^{'}}(x_i)$ as the following:
$$\hat{Q}^{\xi\xi^{'}}(x_i) = -\hat{\mu}^{\xi}(x_i)\hat{\mu}^{\xi^{'}}(x_i)$$
In addition to the index-dependent terms listed above, there is also a 1-electron electronic dipole contribution, scaled by the scalar equivalent to $\lambda\cdot\mu_{nuc}-\lambda\cdot\langle\mu\rangle$.
***

***
### *Augmentation of HF Routine Using CQED Terms
Having completely defined our Hamiltonian, we can now observe the modifications that the CQED approach makes to the canonical RHF procedure. We begin by defining our Fock matrix as:
$$F_{\mu\nu} = H_{\mu\nu} + G_{\mu\nu}\tag{13}$$
$H_{\mu\nu}$ consists of the core Hamiltonian, $h_{\mu\nu}$, (which is equivalent to the potential energy plus the kinetic energy) with both of the 1-electron terms added onto it. $G_{\mu\nu}$ consists of two electron integrals multiplied by the density matrix, D, with our CQED 2-electron terms added onto it. 
Therefore, we get:
$$H_{\mu\nu} = h_{\mu\nu}-\frac{1}{2}\sum_{\xi,\xi^{'}}\lambda^{\xi}\lambda^{\xi^{'}}Q_{\mu\nu}^{\xi\xi^{'}}+(\lambda\cdot\mu_{nuc}-\lambda\cdot\langle\mu\rangle)\sum_{\xi}\lambda^{\xi}\mu_{\mu\nu}^{\xi}\tag{14}$$
$$G_{\mu\nu}=(2(\mu\nu|\lambda\sigma)-(\mu\lambda|\nu\sigma))D_{\lambda\sigma}+\big(\sum_{\xi\xi^{'}}\lambda^{\xi}\lambda^{\xi^{'}}(2\mu_{\mu\nu}^{\xi}\mu_{\lambda\sigma}^{\xi^{'}}-\mu_{\mu\lambda}^{\xi}\mu_{\nu\sigma}^{\xi^{'}})\big)D_{\lambda\sigma}\tag{15}$$

Given these two formulas, we can construct a new formulation for the energy related to our CQED modificaiton:
$$E_{CQED-RHF} = (F_{\mu\nu}+H_{\mu\nu})D_{\mu\nu} + V_{NN}+d_c\tag{16}$$
Where we take $d_c$ to be equal to:
$$d_c = \frac{1}{2}(\lambda\cdot\mu_{nuc})^2 - (\lambda\cdot\langle\mu\rangle)(\lambda\cdot\mu_{nuc})+\frac{1}{2}(\lambda\cdot\langle\mu\rangle)^2\tag{17}$$
Again, Equation 16 follows from the typical HF procedure, where we can utilize the following expression from the HF procedure to express our energy in a more convenient manner, based around distributing the density matrix to our one-electron core Hamiltonian, $H$, as well as to our Fock Matrix, $F$:
$$E = \sum(H + F)D\tag{18}$$

In addition, our CQED-RHF energy value also contains two additional terms, $V_{N,N}$ and $d_{c}$, which represnt the total repulsive coulomb potential of all nuclei $(V_{N,N})$, as well as a constant term that arises from the consideration of our Dipole Self Energy Hamiltonian. 
***
***
### *Outline of the Procedure
Now that we have addressed the theory concerning our CQED approach, we will briefly discuss the procedure for calculating energy values using the CQED-RHF approach. This procedure will be demonstrated in a step-wise manner in the upcoming code segment--we are simply outlining the procedure here.
* Begin by preparing the necessary terms: kinetic, nuclear attraciton, electron repulsion, dipole, quadrupole -- All of which we are going to gather in the atomic orbital basis via usage of the psi4 framework for simplicity
* We simulate the initial loop of the Hartree-Fock procedure, ending with an initial density matrix guess--contracted from coefficients gathered from psi4. The potential energy, kinetic energy, and electron repulsion integrals are additionally gathered in this step, as they will be needed for our calculations.
* We initialize the CQED aspect of our procedure, preparing the all necessary terms for our adaptation of the RHF procedure. More specifically, we are leveraging psi4 to prepare $\langle\mu\rangle$ which is developed through gathering twice $\mu_{(x,y,z)}$ and multiplying it against our guess density matrix. To this, we add $\mu_{nuc}$ terms--which, again are gathered through psi4-- in respect to $(x,y,z)$. Afterwards, we prepare our polarization-scaled term through summing up component wise products between our polarization vector, $\lambda$ and our component differentiated $\mu$ terms. We continue this trend, describing our $\langle\mu\rangle$ and $\mu_{nuc}$ terms respectively scaled by $\lambda$
* We prepare our dipole energy term, $d_c$, using the same equation presented in Equation 17. In addition to this, we prepare and subsequently uitlize dipole and quadrupole terms in order to augment our core Hamiltonian. Likewise, we augment our Fock matrix with terms gathered via contracting 2 electron dipole integrals with our guess density matrix
* From here, we calculate our SCF energy, storing it for future reference as the SCF iteration cycle begins
* Next, we diagonalize the Fock matrix, updating our denisty matrix from the new coefficient matrix. This new density matrix is used to update our terms. 
* Finally, we check for congergence in regards to our calculated energy and density matrix, repeating the process should convergence not be met




## II. Implementation

Using the above overview, let's write a RHF program using <span style="font-variant: small-caps"> Psi4 </span> and NumPy.  First, we need to import these Python modules: 

In [2]:
# ==> Import Psi4 & NumPy <==
import psi4
import numpy as np
import scipy.linalg as la
import time
from helper_cqed_rhf import cqed_rhf



*(This block appears in the basic RHF tutorial... do we need it?)*

Next, using what you learned in the previous tutorial module, set the following <span style="font-variant: small-caps"> Psi4 </span> and molecule options.

Memory & Output specifications:
- Give 500 Mb of memory to Psi4
- Set Psi4 output file to "output.dat"
- Set a variable `numpy_memory` to an acceptable amount of available memory for the working computer to use for storing tensors

Molecule definition:
*(Do we want to do formaldehyde instead?)*

- Define the "physicist's water molecule" (O-H bond length = 1.1 Angstroms, HOH bond angle = 104 degrees)
- Molecular symmetry C1

Computation options:
- basis set cc-pVDZ
- SCF type PK
- Energy convergence criterion to 0.00000001


In [18]:
# ==> Set Basic Psi4 Options <==
# Memory specification
psi4.set_memory(int(5e8))
numpy_memory = 2

# Set output file
psi4.core.set_output_file('output.dat', False)

# Define Physicist's water -- don't forget C1 symmetry!
#molecule_string = """
#O
#H 1 1.1
#H 1 1.1 2 104
#symmetry c1
#"""

#options_dict = {
#    'basis': 'cc-pvdz',
#    'scf_type': 'pk',
#    'e_convergence': 1e-8
#}


molecule_string = """
Mg
H 1 2.2
symmetry c1
1 1
"""

# options dict
options_dict = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}



psi4.set_options(options_dict)

mol = psi4.geometry(molecule_string)
scf_e, wfn = psi4.energy('scf', return_wfn=True)
print(scf_e)

# electric field for H2O - polarized along z-axis with mangitude 0.05 atomic units
lam_h2o = np.array([0.0, 0.0, 0.0125])
lambda_vector = np.copy(lam_h2o)
# run cqed_rhf method
cqed_rhf_dict = cqed_rhf(lam_h2o, molecule_string, options_dict)

omega_val = 4.75 / psi4.constants.Hartree_energy_in_eV



-199.86395910419174

Start SCF iterations:

Canonical RHF One-electron energy = -281.6658725094007423
CQED-RHF One-electron energy      = -281.6660217366671191
Nuclear repulsion energy          = 2.8864211491090908
Dipole energy                     = 0.0009465637014397
SCF Iteration   1: Energy = -199.8633303817348406   dE = -1.99863E+02   dRMS = 2.52850E-05
SCF Iteration   2: Energy = -199.8633315502318339   dE = -1.16850E-06   dRMS = 9.55045E-06
SCF Iteration   3: Energy = -199.8633316895821395   dE = -1.39350E-07   dRMS = 3.61517E-06
SCF Iteration   4: Energy = -199.8633317095171549   dE = -1.99350E-08   dRMS = 1.37921E-06
SCF Iteration   5: Energy = -199.8633317124265432   dE = -2.90939E-09   dRMS = 5.28065E-07
SCF Iteration   6: Energy = -199.8633317128551425   dE = -4.28599E-10   dRMS = 2.02751E-07
SCF Iteration   7: Energy = -199.8633317129182956   dE = -6.31530E-11   dRMS = 7.79875E-08
SCF Iteration   8: Energy = -199.8633317129277884   dE = -9.49285E-12   dRMS = 3.00308E-08
SC

Since we will be writing our own, iterative RHF procedure, we will need to define options that we can use to tweak our convergence behavior.  For example, if something goes wrong and our SCF doesn't converge, we don't want to spiral into an infinite loop.  Instead, we can specify the maximum number of iterations allowed, and store this value in a variable called `maxiter`.  Here are some good default options for our program:
~~~python
MAXITER = 40
E_conv = 1.0e-6
~~~
These are by no means the only possible values for these options, and it's encouraged to try different values and see for yourself how different choices affect the performance of our program.  For now, let's use the above as our default.

In [19]:


# grab necessary quantities from cqed_rhf_dict
scf_e = cqed_rhf_dict["RHF ENERGY"]
cqed_scf_e = cqed_rhf_dict["CQED-RHF ENERGY"]
wfn = cqed_rhf_dict["PSI4 WFN"]
C = cqed_rhf_dict["CQED-RHF C"]
eps = cqed_rhf_dict["CQED-RHF EPS"]
cqed_rhf_dipole_moment = cqed_rhf_dict["CQED-RHF DIPOLE MOMENT"]

# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())

# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc = wfn.nalpha()

# total number of orbitals
nmo = wfn.nmo()

# number of virtual orbitals
nvirt = nmo - ndocc

# need to update the Co and Cv core matrix objects so we can
# utlize psi4s fast integral transformation!

# collect rhf wfn object as dictionary
wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# update wfn_dict with orbitals from CQED-RHF
wfn_dict["matrix"]["Ca"] = C
wfn_dict["matrix"]["Cb"] = C
# update wfn object
wfn = psi4.core.Wavefunction.from_file(wfn_dict)

# occupied orbitals as psi4 objects but they correspond to CQED-RHF orbitals
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals same way
Cv = wfn.Ca_subset("AO", "VIR")

# 2 electron integrals in CQED-RHF basis
ovov = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

# build the (oo|vv) integrals:
oovv = np.asarray(mints.mo_eri(Co, Co, Cv, Cv))

# strip out occupied orbital energies, eps_o spans 0..ndocc-1
eps_o = eps[:ndocc]

# strip out virtual orbital energies, eps_v spans 0..nvirt-1
eps_v = eps[ndocc:]

# Extra terms for Pauli-Fierz Hamiltonian
# nuclear dipole
mu_nuc_x = mol.nuclear_dipole()[0]
mu_nuc_y = mol.nuclear_dipole()[1]
mu_nuc_z = mol.nuclear_dipole()[2]

# l \cdot \mu_nuc for d_c
l_dot_mu_nuc = lambda_vector[0] * mu_nuc_x
l_dot_mu_nuc += lambda_vector[1] * mu_nuc_y
l_dot_mu_nuc += lambda_vector[2] * mu_nuc_z

# dipole arrays in AO basis
mu_ao_x = np.asarray(mints.ao_dipole()[0])
mu_ao_y = np.asarray(mints.ao_dipole()[1])
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform dipole array to CQED-RHF basis
mu_cmo_x = np.dot(C.T, mu_ao_x).dot(C)
mu_cmo_y = np.dot(C.T, mu_ao_y).dot(C)
mu_cmo_z = np.dot(C.T, mu_ao_z).dot(C)

# \lambda \cdot < \mu >
# e.g. line 6 of Eq. (18) in [McTague:2021:ChemRxiv]
l_dot_mu_exp = 0.0
for i in range(0, 3):
    l_dot_mu_exp += lambda_vector[i] * cqed_rhf_dipole_moment[i]

# \lambda \cdot \mu_{el}
# e.g. line 4 Eq. (18) in [McTague:2021:ChemRxiv]
l_dot_mu_el = lambda_vector[0] * mu_cmo_x
l_dot_mu_el += lambda_vector[1] * mu_cmo_y
l_dot_mu_el += lambda_vector[2] * mu_cmo_z

# dipole constants to add to E_CQED_CIS,
#  0.5 * (\lambda \cdot \mu_{nuc})** 2
#      - (\lambda \cdot <\mu> ) ( \lambda \cdot \mu_{nuc})
# +0.5 * (\lambda \cdot <\mu>) ** 2
# Eq. (14) of [McTague:2021:ChemRxiv]
d_c = (
    0.5 * l_dot_mu_nuc ** 2 - l_dot_mu_nuc * l_dot_mu_exp + 0.5 * l_dot_mu_exp ** 2
)

# check to see if d_c what we have from CQED-RHF calculation
assert np.isclose(d_c, cqed_rhf_dict["DIPOLE ENERGY"])

# create Hamiltonian for elements H[ias, jbt]
Htot = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
Hep = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H1e = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H2e = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
H2edp = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)
Hp = np.zeros((ndocc * nvirt * 2 + 2, ndocc * nvirt * 2 + 2), dtype=complex)




In [ ]:
# elements corresponding to <s|<\Phi_0 | H | \Phi_0>|t>
# Eq. (16) of [McTague:2021:ChemRxiv]
Hp[0, 0] = 0.0
Hp[1, 1] = omega_val


In [ ]:

# elements corresponding to <s|<\Phi_0 | H | \Phi_i^a>|t>
# Eq. (17) of [McTague:2021:ChemRxiv]
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        ia0 = 2 * (i * nvirt + a) + 2
        ia1 = 2 * (i * nvirt + a) + 3
        Hep[0, ia1] = Hep[ia1, 0] = (
            -np.sqrt(omega_val) * l_dot_mu_el[i, A]
        )
        Hep[1, ia0] = Hep[ia0, 1] = (
            -np.sqrt(omega_val) * l_dot_mu_el[i, A]
        ) 



In [ ]:
# elements corresponding to <s|<\Phi_i^a| H | \Phi_j^b|t>
# Eq. (18) of [McTague:2021:ChemRxiv]
for i in range(0, ndocc):
    for a in range(0, nvirt):
        A = a + ndocc
        for s in range(0, 2):
            ias = 2 * (i * nvirt + a) + s + 2

            for j in range(0, ndocc):
                for b in range(0, nvirt):
                    B = b + ndocc
                    for t in range(0, 2):
                        jbt = 2 * (j * nvirt + b) + t + 2
                        # ERIs
                        H2e[ias, jbt] = (
                            2.0 * ovov[i, a, j, b] - oovv[i, j, a, b]
                        ) * (s == t)
                        # 2-electron dipole terms
                        # ordinary
                        H2edp[ias, jbt] += (
                            2.0 * l_dot_mu_el[i, A] * l_dot_mu_el[j, B]
                        ) * (s == t)
                        # exchange
                        H2edp[ias, jbt] -= (
                            l_dot_mu_el[i, j] * l_dot_mu_el[A, B] * (s == t)
                        )
                        # orbital energies from CQED-RHF
                        H1e[ias, jbt] += eps_v[a] * (s == t) * (a == b) * (i == j)
                        H1e[ias, jbt] -= eps_o[i] * (s == t) * (a == b) * (i == j)
                        # photonic and dipole energy term
                        Hp[ias, jbt] += (
                            (omega_val * t) * (s == t) * (i == j) * (a == b)
                        )
                        # bilinear coupling - off-diagonals first
                        Hep[ias, jbt] += (
                            np.sqrt(t + 1)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[i, j]
                            * (s == t + 1)
                            * (a == b)
                        )
                        Hep[ias, jbt] += (
                            np.sqrt(t)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[i, j]
                            * (s == t - 1)
                            * (a == b)
                        )
                        Hep[ias, jbt] -= (
                            np.sqrt(t + 1)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[A, B]
                            * (s == t + 1)
                            * (i == j)
                        )
                        Hep[ias, jbt] -= (
                            np.sqrt(t)
                            * np.sqrt(omega_val / 2)
                            * l_dot_mu_el[A, B]
                            * (s == t - 1)
                            * (i == j)
                        )
                        # now handle diagonal in electronic term
                        if a == b and i == j and s == t + 1:
                            # l dot <mu> term
                            Hep[ias, jbt] += (
                                np.sqrt(t + 1)
                                * np.sqrt(omega_val / 2)
                                * l_dot_mu_exp
                            )
                            # l dot mu terms
                            for k in range(0, ndocc):
                                # sum over occupied indices
                                Hep[ias, jbt] -= (
                                    np.sqrt(t + 1)
                                    * np.sqrt(omega_val / 2)
                                    * l_dot_mu_el[k, k]
                                )

                        # now handle diagonal in electronic term
                        if a == b and i == j and s == t - 1:
                            # l dot <mu> term
                            Hep[ias, jbt] += (
                                np.sqrt(t) * np.sqrt(omega_val / 2) * l_dot_mu_exp
                            )
                            # l dot mu terms
                            for k in range(0, ndocc):
                                # sum over occupied indices
                                Hep[ias, jbt] -= (
                                    np.sqrt(t)
                                    * np.sqrt(omega_val / 2)
                                    * l_dot_mu_el[k, k]
                                )





In [ ]:
# Form Htot from sum of all terms
Htot = Hp + Hep + H1e + H2e + H2edp


ECIS, L_CCIS = np.linalg.eigh(Htot)
R_CCIS = np.copy(L_CCIS)

cqed_cis_dict = {
    'RHF ENERGY' : scf_e,
    'CQED-RHF ENERGY' : cqed_scf_e,
    'CQED-CIS ENERGY' : ECIS,
    'CQED-CIS R VECTORS' : R_CCIS,
    'CQED-CIS L VECTORS' : L_CCIS,
    'CQED-CIS H MATRIX' : Htot
}



### References
1. Pauli-Fierz Hamiltonian and CQED-RHF Equations
    - [[Haugland:2020:041043](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.10.041043)] T. S. Haughland, E. Ronco, E. F. Kjonstad, A. Rubio, H. Koch, *Phys. Rev. X*, **10**, 041043 (2020)
    - [[DePrince:2021:094112]](https://aip.scitation.org/doi/10.1063/5.0038748) A. E. DePrince III, *J. Chem. Phys.* **154**, 094113 (2021).
2. Detailed CQED-RHF and CQED-CIS equations and overview of algorithm
    - [[McTague:2021:ChemRxiv](https://chemrxiv.org/engage/chemrxiv/article-details/611fa8d08a6faa13229c8be6)] J. McTague, J. J. Foley IV, *ChemRxiv*,
doi: 10.33774/chemrxiv-2021-0gpz8 (2021)

In [20]:
print(cqed_cis_dict['CQED-CIS ENERGY'][1])

0.16563135967439932
